In [11]:
import os
from PIL import Image
import torch
from torchvision import transforms
from torcheval.metrics import FrechetInceptionDistance

In [12]:
# Function to load images from a folder, including images in subfolders
def load_images_from_folder(folder):
    images = []
    for root, _, files in os.walk(folder):  # Traverse the directory including subdirectories
        for filename in files:
            if filename.endswith(".png"):
                img_path = os.path.join(root, filename)
                img = Image.open(img_path).convert('RGB')  # Open the image and convert to RGB
                if img is not None:
                    images.append(img)
    return images

In [13]:
# Load images from two pressure folders
true_folder_p = ["true_dir"]
fake_folder_p = ["fake_dir"]

# Load images from two velocity folders
true_folder_v = ["true_dir"]
fake_folder_v = ["fake_dir"]

# Check if CUDA is available and set the device
device = torch.device("cuda:id")

In [14]:
images1_p = load_images_from_folder(true_folder_p)
images2_p = load_images_from_folder(fake_folder_p)

# Transform images to tensors
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

images1_p = [transform(img).to(device) for img in images1_p]
images2_p = [transform(img).to(device) for img in images2_p]

# Initialize the FID metric and move to the device
fid_metric = FrechetInceptionDistance().to(device)

# Update the metric with images from the first folder (real images)
for img in images1_p:
    fid_metric.update(img.unsqueeze(0), is_real=True)

# Update the metric with images from the second folder (generated images)
for img in images2_p:
    fid_metric.update(img.unsqueeze(0), is_real=False)

# Compute the FID score
fid_score_pressure = fid_metric.compute()
print("FID Score:", fid_score_pressure)

FID Score: tensor(9.9639, device='cuda:2')


In [15]:
images1_v = load_images_from_folder(true_folder_v)
images2_v = load_images_from_folder(fake_folder_v)

# Transform images to tensors
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

images1_v = [transform(img).to(device) for img in images1_v]
images2_v = [transform(img).to(device) for img in images2_v]

# Initialize the FID metric and move to the device
fid_metric = FrechetInceptionDistance().to(device)

# Update the metric with images from the first folder (real images)
for img in images1_v:
    fid_metric.update(img.unsqueeze(0), is_real=True)

# Update the metric with images from the second folder (generated images)
for img in images2_v:
    fid_metric.update(img.unsqueeze(0), is_real=False)

# Compute the FID score
fid_score_velocity = fid_metric.compute()
print("FID Score:", fid_score_velocity)

NameError: name 'true_folder_v' is not defined

In [ ]:
avg_fid = (fid_score_pressure + fid_score_velocity) / 2
avg_fid

tensor(16.9487, device='cuda:2')